In [0]:
%run "/Workspace/Users/dbxdeschrtrial2@gmail.com/databricks/Other/common"


# read bonze traffic

In [0]:
dbutils.widgets.text(name="env",defaultValue="",label=" Enter the environment in lower case")
global env
env = dbutils.widgets.get("env")

In [0]:

def read_BronzetrafficTable(environment):
    print('Reading the Bronze Table raw_traffic Data : ',end='')
    df_bronzeRoads = (spark.readStream
                    .table(f"`{environment}_catalog`.`bronze`.raw_traffic")
                    )
    print(f'Reading {environment}_catalog.bronze.raw_traffic Success!')
    print("**********************************")
    return df_bronzeRoads

# Transformation create columns

# Creating columns total of electric vehicules

In [0]:
def traffic_Total_EV(df):
    print('Creating traffic Total EV : ', end='')
    from pyspark.sql.functions import when,col

    df_traffic_Cat = df.withColumn("Total_EV",
                                col('EV_car') +  col('EV_bike')
    )
    print(f'Creating traffic Total EV  Success!')
    return df_traffic_Cat

# Creating columns total all motor vehicules

# WRITE to silver.Silver_roads

In [0]:
def write_traffic_SilverTable(StreamingDF,environment):
    print('Writing the silver_traffic Data : ',end='') 

    write_TrafficStreamSilver_R = (StreamingDF.writeStream
                .format('delta')
                .option("checkpointLocation",f'{checkpoint_path}SilvertrafficLoad/raw_traffic/') 
                .outputMode('append')
                .queryName("SilverTrafficWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}_catalog`.`silver`.`silver_traffic`"))
    
    write_TrafficStreamSilver_R.awaitTermination()
    
    print(f'Writing `{environment}_catalog`.`silver`.`silver_traffic` Success!')

# Main

In [0]:
# Initiate variable path
Initglobalvarpath(env)

# read Bronze Table
df_traffic = read_BronzetrafficTable(env)

# Transformation silver layer
# Cleaning Data
df_noDups = remove_Dups(df_traffic)
AllColumns = df_noDups.schema.names
df_clean = handle_NULLs(df_noDups,AllColumns)

# Transformation silver layer
df_traffic_Total_EV = traffic_Total_EV(df_clean)




## Writing data to silver_traffic table
write_traffic_SilverTable(df_traffic_Total_EV , env)


# query the silver table

In [0]:
%sql
select * from dev_catalog.silver.silver_traffic
limit 100